In [3]:
import finnhub
import os
from pydantic import BaseModel
from dotenv import load_dotenv
from typing import Optional
import time

load_dotenv()

class StockPriceResponse(BaseModel):
    current_price: float
    previous_close: float
    change: Optional[float] = None
    change_percent: Optional[float] = None
    high: Optional[float] = None
    low: Optional[float] = None
    open: Optional[float] = None

def get_stock_price(symbol: str) -> StockPriceResponse:
    """
    Get stock price using Finnhub API
    
    Args:
        symbol: Stock ticker symbol (e.g., 'AAPL', 'GOOGL')
    
    Returns:
        StockPriceResponse with current and previous close prices
    
    Raises:
        Exception: If API call fails or returns invalid data
    """
    # Initialize Finnhub client with API key from .env
    api_key = os.getenv('FINHUB_API_KEY')
    if not api_key:
        raise ValueError("FINHUB_API_KEY not found in .env file")
    
    finnhub_client = finnhub.Client(api_key=api_key)
    
    try:
        # Get quote data for the symbol
        quote = finnhub_client.quote(symbol.upper())
        
        # Check if we got valid data
        if not quote or quote.get('c') is None:
            raise ValueError(f"No data available for symbol: {symbol}")
        
        # Extract the data
        current_price = quote['c']  # Current price
        previous_close = quote['pc']  # Previous close
        
        # Create response with additional data if available
        return StockPriceResponse(
            current_price=current_price,
            previous_close=previous_close,
            change=quote.get('d'),  # Change ($)
            change_percent=quote.get('dp'),  # Change (%)
            high=quote.get('h'),  # Day high
            low=quote.get('l'),  # Day low
            open=quote.get('o')  # Day open
        )
        
    except Exception as e:
        raise Exception(f"Failed to fetch stock price for {symbol}: {str(e)}")


In [2]:
# RIVN - Rivian Automotive, Inc.
# LCID - Lucid Group, Inc.
# TSLA - Tesla, Inc.

get_stock_price("VOO") 

StockPriceResponse(current_price=604.44, previous_close=604.57, change=-0.13, change_percent=-0.0215, high=605.98, low=603.95, open=604.59)

In [4]:
# Send SMS with Twilio
from twilio.rest import Client


account_sid = os.getenv("ACCOUNT_SID")
auth_token = os.getenv("AUTH_TOKEN")
to_phone_number = os.getenv("TO_PHONE_NUMBER")
from_phone_number = os.getenv("FROM_PHONE_NUMBER")
client = Client(account_sid, auth_token)

def send_sms( body: str) -> str:
    """
    Send SMS using Twilio API
    
    Args:
        to: Recipient phone number (in E.164 format, e.g., '+1234567890')
        body: Message body
        from_: Sender phone number (Twilio number)
    
    Returns:
        Message SID if sent successfully
    
    Raises:
        Exception: If sending fails
    """
    try:
        message = client.messages.create(
            body=body,
            from_=from_phone_number,
            to=to_phone_number
        )
        return message.sid
    except Exception as e:
        raise Exception(f"Failed to send SMS to {to_phone_number}: {str(e)}")


In [5]:
# Test sending SMS
send_sms("Hello from Twilio and Python!")

'SM0602ec1cbbe52660530d1b6d7bb55063'

In [4]:
import json

def get_tracker_list():
    with open("stocks_lists.json", "r") as f:
        tracker_list = json.load(f)

    return tracker_list

In [5]:
get_tracker_list()

['RIVN', 'LCID', 'TSLA', 'VOO']

In [6]:
from agents import Agent, FileSearchTool, Runner, WebSearchTool, function_tool

@function_tool
async def fetch_stock_price(symbol: str) -> StockPriceResponse:
    return get_stock_price(symbol)


@function_tool
async def send_stock_sms( body: str) -> str:
    return send_sms(body)

@function_tool
async def get_tracker_list_tool() -> dict:
    return get_tracker_list()

In [7]:
from agents import Agent

stock_research_agent = Agent(
  name="Stock Research Agent",
  instructions="""
  You are a Stock Market Researcher. Your job is to research the current news around a specific stock and determine what may have caused recent price movements in the past 24 hours specifically. You have access to be able to retreive the stock price and research via the internet using your tools.

  Use the get_tracker_list tool to retrieve the list of stocks being tracked. Make sure you have an explanation for the movement in the stock price.

  You should use the fetch_stock_price tool to check current stock price, previous close, and other relevant data. Use the WebSearchTool to find recent news articles or information about the stock.

  Output: Your final_output should be a the price, an short analysis of the price change and short summary of your findings, send the summary of each stock via SMS in a unique message to the user using the send_stock_sms tool as the body parameter.
  """,
  tools=[
    fetch_stock_price,
    WebSearchTool(),
    send_stock_sms,
    get_tracker_list_tool
  ],
  model="gpt-4.1"
)

In [8]:
from agents import Runner

async def main():
    result = await Runner.run(stock_research_agent, "Research the stocks in the tracker list and send an SMS with your findings")
    print(result.final_output)

In [9]:

import asyncio

if __name__ == "__main__":
    await main()

Final Output (sent as SMS):

1. Rivian Automotive Inc. (RIVN) closed at $13.46, down 3.65% in the past 24 hours. The decline follows news of workforce reductions ahead of the R2 SUV launch and volatility after a major joint venture with Volkswagen. Market sentiment is cautious amid restructuring.

2. Lucid Group Inc. (LCID) finished at $19.28, falling 3.14%. The price drop is likely related to recent layoffs and general EV sector pressure. No major company-specific news, but industry headwinds persist.

3. Tesla Inc. (TSLA) surged 7.36% to $395.94. The sharp rise is linked to a record compensation proposal for CEO Elon Musk and optimism on Tesla’s AI/robotics future. The board’s support boosts investor sentiment.

4. Vanguard S&P 500 ETF (VOO) closed at $604.44, down 0.02%. The ETF remains stable, reflecting minor market moves and a slight slip due to large-cap tech stocks like NVIDIA. General market sentiment is steady.

These summaries have been sent via SMS, with each stock's report